In [13]:
import os
import re
import shutil
import json
import csv
from pathlib import Path

def scan_scripts(directory):
    """Recursively scan all .slurm and .submit scripts in the current directory and report necessary changes."""
    report = []
    slurm_files = list(Path(directory).rglob("*.slurm"))
    submit_files = list(Path(directory).rglob("*.submit"))
    
    for file in slurm_files:
        report.extend(check_script(file, is_slurm=True))
    
    for file in submit_files:
        report.extend(check_script(file, is_slurm=False))
    
    return report, slurm_files

def check_script(file, is_slurm):
    """Check individual script for necessary changes."""
    with open(file, "r") as f:
        content = f.readlines()
    
    changes = {
        "file": str(file),
        "is_slurm_file": is_slurm,
        "replace_J": False,
        "missing_J_out_err": False,
        "details": []
    }
    
    # Check if -J is present
    for i, line in enumerate(content):
        if re.search(r'\s-J\s', line):
            changes["replace_J"] = True
            changes["details"].append(f"Line {i+1}: '-J' should be replaced with '--job-name'")
        if re.search(r'#SBATCH\s+.*\.(out|err)(?!%J)', line):
            changes["missing_J_out_err"] = True
            changes["details"].append(f"Line {i+1}: Output/Error file directive missing '%J'")
    
    return [changes] if any(changes.values()) else []

def apply_changes(report):
    """Apply necessary changes to scripts based on the report."""
    for change in report:
        file_path = Path(change["file"])
        with open(file_path, "r") as f:
            content = f.readlines()
        
        new_content = []
        
        for line in content:
            # Replace -J with --job-name
            line = re.sub(r'(#SBATCH\s+)-J(\s+)', r'\1--job-name\2', line)
            
            # Append %J to .out and .err files if missing
            line = re.sub(r'(#SBATCH\s+.*\.(out|err))(?!%J)', r'\1%J', line)
            
            new_content.append(line)
        
        # Write changes back
        with open(file_path, "w") as f:
            f.writelines(new_content)

def save_report_to_csv(report, filename="script_reformat_report.csv"):
    """Save the report to a CSV file."""
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["File", "Is SLURM File", "Replace -J", "Missing %J in .out/.err", "Details"])
        for entry in report:
            writer.writerow([entry["file"], entry["is_slurm_file"], entry["replace_J"], entry["missing_J_out_err"], " | ".join(entry["details"])])


In [14]:
# main
# directory = "./JOB-EXAMPLES"
# directory = Path(__file__).parent  # Set directory to the script's location; script already in "./JOB-EXAMPLES"
directory = './'

In [15]:
directory = Path(__file__).parent  # Set directory to the script's location
report, slurm_files = scan_scripts(directory)
print("Report:")
print(json.dumps(report, indent=4))  # Pretty-print the report
print("List of .slurm files:")
print(json.dumps([str(f) for f in slurm_files], indent=4))  # Pretty-print the list of .slurm files
save_report_to_csv(report)
print("Report saved to script_reformat_report.csv")

Renamed files:
[
    [
        "matlab\\parallel.slurm",
        "matlab\\parallel.submit"
    ],
    [
        "matlab\\serial.slurm",
        "matlab\\serial.submit"
    ],
    [
        "matlab_gurobi\\matlab_gurobi.slurm",
        "matlab_gurobi\\matlab_gurobi.submit"
    ],
    [
        "gaussian\\09\\run-g09-general.slurm",
        "gaussian\\09\\run-g09-general.submit"
    ],
    [
        "gaussian\\16\\run-g16-general.slurm",
        "gaussian\\16\\run-g16-general.submit"
    ],
    [
        "ipi_cp2k\\inet_socket\\ipi_cp2k.slurm",
        "ipi_cp2k\\inet_socket\\ipi_cp2k.submit"
    ],
    [
        "ipi_cp2k\\unix_socket\\ipi_cp2k.slurm",
        "ipi_cp2k\\unix_socket\\ipi_cp2k.submit"
    ],
    [
        "ipi_lammps\\unix_socket\\ipi_lammps.slurm",
        "ipi_lammps\\unix_socket\\ipi_lammps.submit"
    ]
]
All .slurm files have been renamed to .submit
Report:
[
    {
        "file": "abinit\\abinit.submit",
        "replace_J": false,
        "missing_J_out_err": true

PermissionError: [Errno 13] Permission denied: 'script_reformat_report.csv'